In [ ]:
import json 

with open('natural_question.json') as f: 
    data = json.load(f)

In [ ]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

print('done')

done


In [ ]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [ ]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

In [ ]:
def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = table_data
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python -m tapas.run_task_main \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/testing/covid-testing-all-observations.csv")
df = df.query("Entity == 'Mauritania - tests performed'")
df = df[['Entity', 'Date', 'Cumulative total']]
df = df.rename(columns = {"Cumulative total" : 'total tests'})
df

,Entity,Date,total tests
27638,Mauritania - tests performed,2020-04-01,63.0
27639,Mauritania - tests performed,2020-04-02,67.0
27640,Mauritania - tests performed,2020-04-03,NaN
27641,Mauritania - tests performed,2020-04-04,NaN
27642,Mauritania - tests performed,2020-04-05,79.0
...,...,...,...
27966,Mauritania - tests performed,2021-03-31,NaN
27967,Mauritania - tests performed,2021-04-01,NaN
27968,Mauritania - tests performed,2021-04-02,NaN
27969,Mauritania - tests performed,2021-04-03,NaN


In [ ]:
date_arr = []
for i in df['Date']: 
    date_arr.append(i)
if '2021-01-07' in date_arr:
    print("test")

In [ ]:
count = 0
question_output = {}
skipped_q = []
is_2021 = True
for i in range(len(data)): 
    if data[str(i+1)][0]['question_template_id'] == '6q1':
        q = data[str(i+1)][0]['question']
        ent = data[str(i+1)][0]['entities']
        if '1st, January' in q or '2nd, January' in q or '3rd, January' in q or '4th, January' in q or '5th, January' in q or '6th, January' in q:
            continue
        if "Cameroon" in q: 
            continue
        print(count)
        print(q)
        print(ent)
        if ent[2] == 'January': 
            df2 = df.iloc[245:267]
        elif ent[2] == 'February': 
            df2 = df.iloc[270:298]
        elif ent[2] == 'March': 
            df2 = df.iloc[298:326]
        elif ent[2] == 'April':
            df2 = df.iloc[0:28]
        elif ent[2] == 'May': 
            df2 = df.iloc[30:58]
        elif ent[2] == 'June': 
            df2 = df.iloc[61:89]
        elif ent[2] == 'July': 
            df2 = df.iloc[91:119]
        elif ent[2] == 'August':
            df2 = df.iloc[122:150]
        df2 = df2.astype(str)
        list_of_list = [[]]
        list_of_list[0] = list(df2.columns)
        list_of_list.extend(df2.values.tolist())
        list_of_list
        result = predict(list_of_list, [q])
        res = ""
        count1 = 0
        print(result)
        for i in result[0]: 
            if count1 == 0:
                count1 +=1 
                res += list_of_list[i[0]+1][i[1]]
            else: 
                res += ', ' + list_of_list[i[0]+1][i[1]]
            question_output[q] = res
        count += 1
print(count)
print(question_output)
question_output

0
What are the number of total tests done by Mauritania in 7th, January?
['total tests', 'Mauritania', 'January']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Entity,Date,total tests
Mauritania - tests performed,2021-01-07,160655.0
Mauritania - tests performed,2021-01-08,nan
Mauritania - tests performed,2021-01-09,nan
Mauritania - tests performed,2021-01-10,164521.0
Mauritania - tests performed,2021-01-11,165716.0
Mauritania - tests performed,2021-01-12,nan
Mauritania - tests performed,2021-01-13,nan
Mauritania - tests performed,2021-01-14,170574.0
Mauritania - tests performed,2021-01-15,nan
Mauritania - tests performed,2021-01-16,nan



> What are the number of total tests done by Mauritania in 7th, January?
nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0
[{(11, 2), (21, 2), (16, 2), (6, 2), (7, 2), (1, 2), (12, 2), (17, 2), (2, 2), (3, 2), (13, 2), (8, 2), (18, 2), (19, 2), (4, 2), (14, 2), (15, 2), (9, 2), (20, 2), (5, 2), (0, 2), (10, 2)}]
1
What are the number of total tests done by Mauritania in 8th, January?
['total tests', 'Mauritania', 'January']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Entity,Date,total tests
Mauritania - tests performed,2021-01-07,160655.0
Mauritania - tests performed,2021-01-08,nan
Mauritania - tests performed,2021-01-09,nan
Mauritania - tests performed,2021-01-10,164521.0
Mauritania - tests performed,2021-01-11,165716.0
Mauritania - tests performed,2021-01-12,nan
Mauritania - tests performed,2021-01-13,nan
Mauritania - tests performed,2021-01-14,170574.0
Mauritania - tests performed,2021-01-15,nan
Mauritania - tests performed,2021-01-16,nan



> What are the number of total tests done by Mauritania in 8th, January?
nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0
[{(11, 2), (21, 2), (16, 2), (6, 2), (7, 2), (1, 2), (12, 2), (17, 2), (2, 2), (3, 2), (13, 2), (8, 2), (18, 2), (19, 2), (4, 2), (14, 2), (15, 2), (9, 2), (20, 2), (5, 2), (0, 2), (10, 2)}]
2
What are the number of total tests done by Mauritania in 9th, January?
['total tests', 'Mauritania', 'January']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


...

Entity,Date,total tests
Mauritania - tests performed,2020-08-01,nan
Mauritania - tests performed,2020-08-02,63494.0
Mauritania - tests performed,2020-08-03,63874.0
Mauritania - tests performed,2020-08-04,nan
Mauritania - tests performed,2020-08-05,64723.0
Mauritania - tests performed,2020-08-06,65238.0
Mauritania - tests performed,2020-08-07,65578.0
Mauritania - tests performed,2020-08-08,65798.0
Mauritania - tests performed,2020-08-09,65952.0
Mauritania - tests performed,2020-08-10,66169.0



> What are the number of total tests done by Mauritania in 19th, August?
67083.0, nan, 68492.0, nan, 65578.0, nan, 65798.0, 63494.0, 67415.0, nan, nan, 68872.0, 63874.0, nan, 67832.0, 65952.0, 69284.0, nan, nan, 64723.0, 67962.0, 68086.0, 66169.0, nan, nan, 65238.0, nan, 66540.0
[{(11, 2), (21, 2), (16, 2), (26, 2), (6, 2), (27, 2), (7, 2), (1, 2), (12, 2), (22, 2), (23, 2), (17, 2), (2, 2), (3, 2), (13, 2), (8, 2), (18, 2), (19, 2), (24, 2), (4, 2), (14, 2), (15, 2), (9, 2), (20, 2), (25, 2), (5, 2), (0, 2), (10, 2)}]
200
What are the number of total tests done by Mauritania in 20th, August?
['total tests', 'Mauritania', 'August']
is_built_with_cuda: True
is_gpu_available: False
GPUs: []
Training or predicting ...
Evaluation finished after training step 0.


Entity,Date,total tests
Mauritania - tests performed,2020-08-01,nan
Mauritania - tests performed,2020-08-02,63494.0
Mauritania - tests performed,2020-08-03,63874.0
Mauritania - tests performed,2020-08-04,nan
Mauritania - tests performed,2020-08-05,64723.0
Mauritania - tests performed,2020-08-06,65238.0
Mauritania - tests performed,2020-08-07,65578.0
Mauritania - tests performed,2020-08-08,65798.0
Mauritania - tests performed,2020-08-09,65952.0
Mauritania - tests performed,2020-08-10,66169.0



> What are the number of total tests done by Mauritania in 20th, August?
67083.0, nan, 68492.0, nan, 65578.0, nan, 65798.0, 63494.0, 67415.0, nan, nan, 68872.0, 63874.0, nan, 67832.0, 65952.0, 69284.0, nan, nan, 64723.0, 67962.0, 68086.0, 66169.0, nan, nan, 65238.0, nan, 66540.0
[{(11, 2), (21, 2), (16, 2), (26, 2), (6, 2), (27, 2), (7, 2), (1, 2), (12, 2), (22, 2), (23, 2), (17, 2), (2, 2), (3, 2), (13, 2), (8, 2), (18, 2), (19, 2), (24, 2), (4, 2), (14, 2), (15, 2), (9, 2), (20, 2), (25, 2), (5, 2), (0, 2), (10, 2)}]
201
{'What are the number of total tests done by Mauritania in 7th, January?': 'nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0', 'What are the number of total tests done by Mauritania in 8th, January?': 'nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0', 'What are the number of tot

{'What are the number of total tests done by Mauritania in 7th, January?': 'nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0',
 'What are the number of total tests done by Mauritania in 8th, January?': 'nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0',
 'What are the number of total tests done by Mauritania in 9th, January?': 'nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0',
 'What are the number of total tests done by Mauritania in 10th, January?': 'nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0',
 'What are the number of total tests done by Mauritania in 11th, January?': 'nan, nan, nan, nan, 170574.0, nan,

In [ ]:
question_output

{'What are the number of total tests done by Mauritania in 7th, January?': 'nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0',
 'What are the number of total tests done by Mauritania in 8th, January?': 'nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0',
 'What are the number of total tests done by Mauritania in 9th, January?': 'nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0',
 'What are the number of total tests done by Mauritania in 10th, January?': 'nan, nan, nan, nan, 170574.0, nan, nan, nan, nan, 164521.0, nan, nan, nan, 183648.0, 165716.0, 178831.0, nan, nan, nan, nan, 160655.0, 173801.0',
 'What are the number of total tests done by Mauritania in 11th, January?': 'nan, nan, nan, nan, 170574.0, nan,

In [ ]:
len(question_output)

201

In [ ]:
with open('6q1newdata.json', 'w') as outfile:
    json.dump(question_output, outfile)